# 6. Advanced RAG


In [ ]:
# google colab 用なのでコメントアウト。
# import os
# from google.colab import userdata

# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
# os.environ["LANGCHAIN_PROJECT"] = "agent-book"
# os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

## 6.2. ハンズオンの準備


In [ ]:
# google colab 用なのでコメントアウト。uvを使っている場合はインストール済み
# !pip install langchain-core==0.3.0 langchain-openai==0.2.0 \
#     langchain-community==0.3.0 GitPython==3.1.43 \
#     langchain-chroma==0.1.4 tavily-python==0.5.0

In [ ]:
from dotenv import load_dotenv

# 事前に .env ファイルを作って、OPENAI_API_KEY, LANGCHAIN_API_KEY, TAVILY_API_KEY, COHERE_API_KEY などを設定してください
load_dotenv()

True

In [5]:
# コンテナ内で git を使う場合は、git checkout時にエラーにならないように以下を実行してください
!git config --global --add safe.directory /workspaces/agent-book/chapter06/langchain

In [7]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

370


In [8]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = (
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースのビルディングブロックやコンポーネント、サードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを使用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用して、チェーンを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換することができます。\n\nLangChainは、以下のオープンソースライブラリで構成されています：\n- `langchain-core`: 基本的な抽象化とLangChain表現言語。\n- `langchain-community`: サードパーティの統合。\n- `langchain`: アプリケーションの認知アーキテクチャを構成するチェーン、エージェント、検索戦略。\n- LangGraph: LLMを使用して堅牢で状態を持つマルチアクターアプリケーションを構築するためのライブラリ。\n- LangServe: LangChainチェーンをREST APIとしてデプロイするためのツール。\n- LangSmith: LLMアプリケーションをデバッグ、テスト、評価、監視するための開発者プラットフォーム。\n\nLangChainは、PythonとJavaScriptの両方のライブラリがあり、特にPythonのLangChainライブラリに焦点を当てたドキュメントが提供されています。'

## 6.3. 検索クエリの工夫


### HyDE（Hypothetical Document Embeddings）


In [10]:
hypothetical_prompt = ChatPromptTemplate.from_template("""\
次の質問に回答する一文を書いてください。

質問: {question}
""")

hypothetical_chain = hypothetical_prompt | model | StrOutputParser()

In [11]:
hyde_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hypothetical_chain | retriever,
    }
    | prompt
    | model
    | StrOutputParser()
)

hyde_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースのビルディングブロックやコンポーネント、サードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築することができます。\n\n2. **生産化**: LangSmithを使用して、チェーンを検査、監視、評価し、アプリケーションを継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイメント**: LangGraphアプリケーションを本番環境向けのAPIやアシスタントに変換することができます。\n\nLangChainは、標準化されたコンポーネントインターフェース、オーケストレーション、可観測性と評価といった主要なニーズに応えることを目指しています。これにより、開発者は異なるプロバイダーを簡単に切り替えたり、複雑なアプリケーションを構築したりすることが可能になります。'

### 複数の検索クエリの生成


In [12]:
from pydantic import BaseModel, Field


class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(..., description="検索クエリのリスト")


query_generation_prompt = ChatPromptTemplate.from_template("""\
質問に対してベクターデータベースから関連文書を検索するために、
3つの異なる検索クエリを生成してください。
距離ベースの類似性検索の限界を克服するために、
ユーザーの質問に対して複数の視点を提供することが目標です。

質問: {question}
""")

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    | (lambda x: x.queries)
)  # fmt: skip

In [13]:
multi_query_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": query_generation_chain | retriever.map(),
    }
    | prompt
    | model
    | StrOutputParser()
)  # fmt: skip

multi_query_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります。\n\n1. **開発**: LangChainのオープンソースのビルディングブロック、コンポーネント、サードパーティ統合を使用してアプリケーションを構築できます。また、LangGraphを使用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを利用して、チェーンを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換することができます。\n\nLangChainは、以下のオープンソースライブラリで構成されています：\n- `langchain-core`: 基本的な抽象化とLangChain表現言語（LCEL）。\n- `langchain-community`: サードパーティ統合。\n- `langchain`: アプリケーションの認知アーキテクチャを構成するチェーン、エージェント、検索戦略。\n- LangGraph: LLMを使用して堅牢で状態を持つマルチアクターアプリケーションを構築するためのライブラリ。\n- LangServe: LangChainチェーンをREST APIとしてデプロイするためのツール。\n- LangSmith: LLMアプリケーションをデバッグ、テスト、評価、監視するための開発者プラットフォーム。\n\nLangChainは、開発者がアプリケーションを簡単に構築できるように設計されており、さまざまなコンポーネントを組み合わせて使用することができます。'

## 6.4. 検索後の工夫


### RAG Fusion


In [14]:
from langchain_core.documents import Document


def reciprocal_rank_fusion(
    retriever_outputs: list[list[Document]],
    k: int = 60,  # k は、ハイパーパラメータ
) -> list[str]:
    # 各ドキュメントのコンテンツ (文字列) とそのスコアの対応を保持する辞書を準備
    content_score_mapping = {}

    # 検索クエリごとにループ
    for docs in retriever_outputs:
        # 検索結果のドキュメントごとにループ
        for rank, doc in enumerate(docs):
            content = doc.page_content

            # 初めて登場したコンテンツの場合はスコアを0で初期化
            if content not in content_score_mapping:
                content_score_mapping[content] = 0

            # (1 / (順位 + k)) のスコアを加算
            content_score_mapping[content] += 1 / (rank + k)

    # スコアの大きい順にソート
    ranked = sorted(content_score_mapping.items(), key=lambda x: x[1], reverse=True)  # noqa
    return [content for content, _ in ranked]

In [15]:
rag_fusion_chain = (
    {
        "question": RunnablePassthrough(),
        "context": query_generation_chain | retriever.map() | reciprocal_rank_fusion,
    }
    | prompt
    | model
    | StrOutputParser()
)  # fmt: skip

rag_fusion_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、開発者がアプリケーションを簡単に構築できるようにすることを目指しており、オープンソースのライブラリやコンポーネント、サードパーティの統合を提供しています。\n\nLangChainの主な特徴には以下が含まれます：\n\n1. **標準化されたコンポーネントインターフェース**: 様々なAIアプリケーションに必要なコンポーネントのインターフェースを標準化し、異なるプロバイダー間での切り替えを容易にします。\n\n2. **オーケストレーション**: 複数のコンポーネントやモデルを組み合わせて複雑なアプリケーションを構築するための効率的な接続を提供します。\n\n3. **可観測性と評価**: アプリケーションの動作を理解し、開発の進行を助けるためのトレーシングや評価機能を提供します。\n\nLangChainは、以下の主要なライブラリで構成されています：\n\n- **`langchain-core`**: 基本的な抽象化とLangChain表現言語（LCEL）。\n- **`langchain-community`**: サードパーティの統合。\n- **`langchain`**: アプリケーションの認知アーキテクチャを構成するチェーン、エージェント、検索戦略。\n- **[LangGraph](https://langchain-ai.github.io/langgraph)**: LLMを用いた堅牢で状態を持つマルチアクターアプリケーションを構築するためのライブラリ。\n- **[LangServe](/docs/langserve)**: LangChainチェーンをREST APIとしてデプロイするためのツール。\n- **[LangSmith](https://docs.smith.langchain.com)**: LLMアプリケーションをデバッグ、テスト、評価、監視するためのプラットフォーム。\n\nLangChainは、開発、運用、デプロイの各段階でのサポートを提供し、開発者が自信を持ってアプリケーションを最適化し、展開できるようにします。'

### Cohere のリランクモデルを使用する準備


In [ ]:
# google colab 用なのでコメントアウト
# 最初に実行した load_dotenv() で読み込み済みの想定
# os.environ["COHERE_API_KEY"] = userdata.get("COHERE_API_KEY")

In [ ]:
# uvを使っている場合はインストール済み
# !pip install langchain-cohere==0.3.0

### Cohere のリランクモデルの導入


In [16]:
from typing import Any

from langchain_cohere import CohereRerank
from langchain_core.documents import Document
from typing import Sequence


def rerank(inp: dict[str, Any], top_n: int = 3) -> Sequence[Document]:
    question = inp["question"]
    documents = inp["documents"]

    # https://docs.cohere.com/v2/docs/rerank-2
    cohere_reranker = CohereRerank(model="rerank-multilingual-v3.0", top_n=top_n)
    return cohere_reranker.compress_documents(documents=documents, query=question)


rerank_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "documents": retriever,
    }
    | RunnablePassthrough.assign(context=rerank)
    | prompt
    | model
    | StrOutputParser()
)

rerank_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります。\n\n1. **開発**: LangChainのオープンソースのビルディングブロックやコンポーネント、サードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用して、チェーンを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換することができます。\n\nLangChainは、以下のオープンソースライブラリで構成されています：\n- `langchain-core`: 基本的な抽象化とLangChain表現言語。\n- `langchain-community`: サードパーティの統合。\n- `langchain`: アプリケーションの認知アーキテクチャを構成するチェーン、エージェント、検索戦略。\n- LangGraph: LLMを使用して堅牢で状態を持つマルチアクターアプリケーションを構築するためのライブラリ。\n- LangServe: LangChainチェーンをREST APIとしてデプロイするためのツール。\n- LangSmith: LLMアプリケーションをデバッグ、テスト、評価、監視するためのプラットフォーム。\n\nLangChainは、開発者がアプリケーションを簡単に構築できるようにすることを目指しており、さまざまなコンポーネントを組み合わせて使用することができます。'

## 6.5. 複数の Retriever を使う工夫


### LLM によるルーティング


In [17]:
from langchain_community.retrievers import TavilySearchAPIRetriever

langchain_document_retriever = retriever.with_config(
    {"run_name": "langchain_document_retriever"}
)  # fmt: skip

web_retriever = TavilySearchAPIRetriever(k=3).with_config(
    {"run_name": "web_retriever"}
)  # fmt: skip


In [18]:
from enum import Enum


class Route(str, Enum):
    langchain_document = "langchain_document"
    web = "web"


class RouteOutput(BaseModel):
    route: Route


route_prompt = ChatPromptTemplate.from_template("""\
質問に回答するために適切なRetrieverを選択してください。

質問: {question}
""")

route_chain = (
    route_prompt
    | model.with_structured_output(RouteOutput)
    | (lambda x: x.route)
)  # fmt: skip

In [19]:
def routed_retriever(inp: dict[str, Any]) -> list[Document]:
    question = inp["question"]
    route = inp["route"]

    if route == Route.langchain_document:
        return langchain_document_retriever.invoke(question)
    elif route == Route.web:
        return web_retriever.invoke(question)

    raise ValueError(f"Unknown retriever: {retriever}")


route_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "route": route_chain,
    }
    | RunnablePassthrough.assign(context=routed_retriever)
    | prompt
    | model
    | StrOutputParser()
)

In [20]:
route_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースのビルディングブロックやコンポーネント、サードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを使用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用して、チェーンを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換することができます。\n\nLangChainは、以下のオープンソースライブラリで構成されています：\n- `langchain-core`: 基本的な抽象化とLangChain表現言語（LCEL）。\n- `langchain-community`: サードパーティの統合。\n- `langchain`: アプリケーションの認知アーキテクチャを構成するチェーン、エージェント、検索戦略。\n- LangGraph: LLMを使用して堅牢で状態を持つマルチアクターアプリケーションを構築するためのライブラリ。\n- LangServe: LangChainチェーンをREST APIとしてデプロイするためのツール。\n- LangSmith: LLMアプリケーションをデバッグ、テスト、評価、監視するための開発者プラットフォーム。\n\nLangChainは、Pythonライブラリに焦点を当てており、JavaScript用のドキュメントも提供されています。'

In [21]:
route_rag_chain.invoke("東京の今日の天気は？")

'文脈には具体的な今日の天気の詳細は記載されていませんが、東京の天気予報に関する情報が含まれています。具体的な天気情報を知りたい場合は、提供されたリンク（例えば、Yahoo!天気や日本気象協会のサイト）を参照してください。'

### ハイブリッド検索の実装


In [ ]:
# uvを使っている場合はインストール済み
# !pip install rank-bm25==0.2.2

In [23]:
from langchain_community.retrievers import BM25Retriever

chroma_retriever = retriever.with_config(
    {"run_name": "chroma_retriever"}
)  # fmt: skip

bm25_retriever = BM25Retriever.from_documents(documents).with_config(
    {"run_name": "bm25_retriever"}
)  # fmt: skip

In [24]:
from langchain_core.runnables import RunnableParallel

hybrid_retriever = (
    RunnableParallel(
        {
            "chroma_documents": chroma_retriever,
            "bm25_documents": bm25_retriever,
        }
    )
    | (lambda x: [x["chroma_documents"], x["bm25_documents"]])
    | reciprocal_rank_fusion
)

In [25]:
hybrid_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hybrid_retriever,
    }
    | prompt
    | model
    | StrOutputParser()
)

hybrid_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります。\n\n1. **開発**: LangChainのオープンソースのビルディングブロックやコンポーネント、サードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを使用して、状態を持つエージェントを構築することも可能です。\n\n2. **生産化**: LangSmithを利用して、チェーンを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換することができます。\n\nLangChainは、以下のオープンソースライブラリで構成されています：\n- `langchain-core`: 基本的な抽象化とLangChain表現言語（LCEL）。\n- `langchain-community`: サードパーティの統合。\n- `langchain`: アプリケーションの認知アーキテクチャを構成するチェーン、エージェント、検索戦略。\n- LangGraph: LLMを使用して堅牢で状態を持つマルチアクターアプリケーションを構築するためのライブラリ。\n- LangServe: LangChainチェーンをREST APIとしてデプロイするためのツール。\n- LangSmith: LLMアプリケーションをデバッグ、テスト、評価、監視するためのプラットフォーム。\n\nLangChainは、開発者がアプリケーションを簡単に構築できるようにすることを目指しており、さまざまなコンポーネントを組み合わせて使用することができます。'